## Monopoly Game Agent (v2)

**Proof of Concept**: Connecting LLM I/O to game simulators

The following notebook contains:
- Function wrappers to retrieve game state
- Basic prompt template that we can inject state into
- Output parser to extract necessary data

Note: You can download and import into Goolge Colab if that is preferred.

---

### Project Setup
Install dependencies

In [ ]:
%pip install -U langchain-community langchain-chroma langchain-openai

Import monopoly simulator

(Note for future: We should install the game simulator as a package)

In [26]:
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

Importing other dependencies

In [27]:
# LangChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI, LLMChain, PromptTemplate

# other imports
from pydantic import BaseModel, Field
from typing import List


### Game Functions
Wrappers that retrieves relevant game state(s)

In [28]:
def initialize_game() -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    player1 = Player('player1', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('player2', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2] # For now, player 1 always comes first
    }

In [29]:
def get_current_state(players) -> dict:
    """
    Retrieves the current state of each player, including position, owned roads, 
    money, mortgaged properties, and other status details.

    Args:
        players (list[Player]): List of Player objects in the game.

    Returns:
        dict: A dictionary containing:
            - "players": A list of dictionaries, each with a player's state.
    """
    
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

In [30]:
# Example usage of the above function
initial_state = initialize_game()
initial_state["bank"]

{'cash': 5000, 'houses': 32, 'hotels': 12}

### Prompt Template

The following defines a customizable prompt template for an agent in a Monopoly game. Each part of the template is easily customizable using placeholders for various game elements.

In [31]:
# The agent plays as Player 1 by default
agent_role = "Player 1" 

In [32]:
# prompt template wrapper / hook, a function that returns a string
def prompt_template():
    """
    Generates a formatted prompt string for an agent in a Monopoly game, detailing 
    the game's current state and guiding strategic decision-making.

    Returns:
        str: A prompt template string with placeholders for:
            - {agent_role}: The role of the agent in the game.
            - {initial_bank}: Initial bank details.
            - {initial_board}: Initial board configuration.
            - {initial_roads}: List of roads.
            - {initial_properties}: List of properties.

    Usage:
        Substitute placeholders to customize the prompt with the game state.
    """
    
    return  """
        You are the {agent_role} in a Monopoly game. Here is the current game state:

        Bank:
        {initial_bank}

        Board:
        {initial_board}

        Roads:
        {initial_roads}

        Properties:
        {initial_properties}

        Players:
        Player 1 and Player 2

        Your Objective:
        Given the current state of the game, make strategic moves that maximizes your chances of winning.

        Guidelines:
        1. Analyze each component of the game state to understand your current situation.
        2. Consider any immediate risks or opportunities from property ownership, player positions, or your current balance.

        Instructions:
        - Reason step-by-step to ensure your action aligns with the game’s rules and overall strategy.
        - Provide your next move by determining if you should buy the property or not (yes or no)
  """

**Sample Usage:**
```python
# Define the game setup and get initial game state
game = initialize_game()  # Initializes the bank, board, roads, properties, and players

# Generate the prompt with specific game details
template = prompt_template()
formatted_prompt = template.format(
    agent_role="Player 1",
    initial_bank=game["bank"],
    initial_board=game["board"],
    initial_roads=game["roads"],
    initial_properties=game["properties"]
)

print(formatted_prompt)
```

### Output Parser

The following defines a parser for interpreting the agent's output in the Monopoly game

In [33]:
class Output(BaseModel):
    reasoning: str = Field(description="Your reasoning for the decision")
    decision: str = Field(description="Your decision for the next move")

In [34]:
def output_parser(model):
    return model.with_structured_output(Output)

### Simulate the Game

Set up prompt template and LLM chain

In [35]:
model = ChatOpenAI(model="gpt-4o")

In [36]:
structured_llm = model.with_structured_output(Output)

Initialize the game and make arbitrary moves

In [37]:
game = initialize_game()

In [38]:
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

stop_at_round = 5 # arbitrary number of rounds to play before agent comes in and make a decision (for POC)

In [39]:
idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < stop_at_round:
    for player in list_players:
        player.play()
    idx_count += 1

injecting variables

1. Set up prompt template and LLM chain
2. Hardcode some injection variables & make sure it works
3. Code to retrieve game info / states

In [40]:
initial_template = prompt_template()

In [41]:
### Only one turn of the game is played so far
context = initial_template.format(
    agent_role="Player 1",  # or as appropriate
    initial_bank=game["bank"],
    initial_board=game["board"],
    initial_roads=game["roads"],
    initial_properties=game["properties"]
)

In [44]:
response = structured_llm.invoke(f"${context}. player_state is ${get_current_state(list_players)}")

In [45]:
response.decision

'yes'

In [46]:
response.reasoning

'Player 1 is currently on Oxford Street (position 32). With a dice roll of 10, they will land on Bond Street (position 34), which is unowned and costs $320. Player 1 has $946 in cash, which is enough to purchase Bond Street and still maintain a healthy balance. Buying Bond Street could be a strategic move as it is part of the green color set, and Player 1 already owns Oxford Street in the same set. Purchasing Bond Street would bring Player 1 closer to completing the green set, which could significantly increase rent income if developed. Additionally, Bond Street has a higher rent value, making it a valuable property for future income.'